In [3]:
import autokeras
from autokeras import TextRegressor
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pysrt
import re
import tqdm
import pandas as pd
import logging
from tqdm import tqdm
text_reg = TextRegressor(overwrite=True, max_trials=1)



In [4]:
def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - removes any single character tokens
    Parameters
    ----------
        message : The text message to be preprocessed
    Returns
    -------
        text: The preprocessed text
    """ 
    # Lowercase the twit message
    text = message.lower()
    # Replace URLs with a space in the message
    text = re.sub('https?:\/\/[a-zA-Z0-9@:%._\/+~#=?&;-]*', ' ', text)
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub('\$[a-zA-Z0-9]*', ' ', text)
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub('\@[a-zA-Z0-9]*', ' ', text)
    # Replace everything not a letter or apostrophe with a space
    text = re.sub('[^a-zA-Z\']', ' ', text)
    # Remove single letter words
    text = ' '.join( [w for w in text.split() if len(w)>1] )
    
    return text
        
# Process for all messages
df = pd.read_csv(r"OCEAN.csv",encoding='latin1')
preprocessed = []
for i in range(0,9499):
    message = df['Content'].iloc[i]
    processed_Text = preprocess(message)
    df.loc[i, 'Content'] = processed_Text
    preprocessed.append(str(processed_Text))


In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split

# Split the data into features and target
X = df['Content'] # assuming that the text data is stored in a column called 'text'
y = df['ope'] # assuming that the target values are stored in a column called 'target'

# Tokenize the text data using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
X_encoded = [tokenizer.encode(text, add_special_tokens=True, max_length=512, truncation=True) for text in X]
X_padded = tf.keras.preprocessing.sequence.pad_sequences(X_encoded, padding='post', maxlen=512)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.1)

# Load the BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Freeze the BERT layers to prevent them from being trained
for layer in bert_model.layers:
    layer.trainable = False

# Build the neural network
inputs = tf.keras.Input(shape=(512,), dtype='int32')
embedding = bert_model(inputs)[0]
flatten = tf.keras.layers.Flatten()(embedding)
dense1 = tf.keras.layers.Dense(128, activation='relu')(flatten)
dense2 = tf.keras.layers.Dense(64, activation='relu')(dense1)
output = tf.keras.layers.Dense(1, activation='linear')(dense2)
model = tf.keras.Model(inputs=inputs, outputs=output)

# Compile the model
model.compile(loss='mse', optimizer='adam')

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=16)


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/3
535/535 [==============================] - 5788s 11s/step - loss: 680.3600 - val_loss: 0.7012
Epoch 2/3
535/535 [==============================] - 6003s 11s/step - loss: 1.1847 - val_loss: 9.3508
Epoch 3/3
535/535 [==============================] - 6329s 12s/step - loss: 4.3314 - val_loss: 3.0020


In [3]:

# Use the model to make predictions on new text data
new_text = ['This is a new text for prediction', 'Another text to predict']
new_text_encoded = [tokenizer.encode(text, add_special_tokens=True, max_length=512, truncation=True) for text in new_text]
new_text_padded = tf.keras.preprocessing.sequence.pad_sequences(new_text_encoded, padding='post', maxlen=512)
new_text_predictions = model.predict(new_text_padded)
print('Predictions for new text: {}'.format(new_text_predictions))


NameError: name 'tokenizer' is not defined

In [4]:
import pickle
pickle.dump(model, open('model_age_nn_bert.pkl', 'wb'))

NameError: name 'model' is not defined

In [ ]:
pickled_model = pickle.load(open('model_age_nn_bert.pkl', 'rb'))
